<a href="https://colab.research.google.com/github/christophermalone/HLA311/blob/main/Module2_Part1_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module 2 | Part 1: SELECT() Action - Advanced Level 

This purpose of this iPython Notebook is to communicate the process by which a data scientist would perform a SELECT() action using Python.

<table width='100%' ><tr><td bgcolor='green'></td></tr></table>

## Example - MN Asthma Rates by County
This example will consider  the asthma rate in the 87 counties in Minnesota.  The data for this example will be obtained from the Minnesota Department of Health Public Data Portal.

*   Asthma Query: https://data.web.health.state.mn.us/web/mndata/asthma_query
*   Data Access Portal (Topics): https://data.web.health.state.mn.us/web/mndata/home


The data processing steps for this example will include:
*   Automatically download the file from the internet
*   Remove all information contained in the footer of the data file
*   Fix the issue related to the missing field, i.e. County field name is missing
*   Load the data into Python
*   Perform a SELECT action to retrieve FIPS, County, Count, Population, and Rate
*   Write out the desired data table so further analyses can be completed

<table width='100%' ><tr><td bgcolor='green'></td></tr></table>

### Step 1: Download the data

Create the script to complete the download the data

In [1]:
#Create a file that contains a set of commands to download file
%%bash
{  
 echo 'wget -O  /content/MN_Asthma_Counties.csv http://www.statsclass.org/online/hla311/datasets/MN_Asthma_Counties.csv'
 } > Download_MNAsthma.sh

Run the script to complete the download

In [2]:
#Complete the actual download of the data
%%bash
bash Download_MNAsthma.sh

--2021-05-25 15:37:48--  http://www.statsclass.org/online/hla311/datasets/MN_Asthma_Counties.csv
Resolving www.statsclass.org (www.statsclass.org)... 192.254.227.17
Connecting to www.statsclass.org (www.statsclass.org)|192.254.227.17|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13492 (13K) [text/csv]
Saving to: ‘/content/MN_Asthma_Counties.csv’

     0K .......... ...                                        100% 98.4M=0s

2021-05-25 15:37:49 (98.4 MB/s) - ‘/content/MN_Asthma_Counties.csv’ saved [13492/13492]



## Step 2: Remove Information in Footer

Taking a look at the footer information in this file -- the last 18 lines need to be deleted

In [6]:
#View the footer contents of the file
!tail -20 '/content/MN_Asthma_Counties.csv'

"27171","Wright","Emergency department visits","2016-2018","All","All ages","670.2","399395","16.8","(15.5 - 18.1)","17.4","(16.0 - 18.7)"
"27173","Yellow Medicine","Emergency department visits","2016-2018","All","All ages","81.2","28968","28.0","(22.3 - 34.8)","32.1","(25.3 - 40.2)"

* To protect an individual's privacy hospitalizations and ED visit counts from 1 to 5 are suppressed if the underlying population is less than or equal to 100000.
UR = Unstable Rate. Rates based on numerators less than or equal to 20 may be unstable and should be interpreted with caution.
** Person-years is equal to the sum of the population for each year.
***Age-adjusted rates can only be calculated for "All ages combined".
Hospitalizations data do not include data from federal and sovereign hospitals (e.g. Veteran's Administration; Indian Health Service) or data on Minnesota residents seen in facilities outside of Minnesota and North Dakota. For more information see About the Asthma Data: http://health.

In [7]:
#Removing the information in footer
%%bash
sed -i -e 90,108d '/content/MN_Asthma_Counties.csv'

In [11]:
#Check to make sure all footer information has been successfully removed
!tail '/content/MN_Asthma_Counties.csv'

"27155","Traverse","Emergency department visits","2016-2018","All","All ages","19.0","9681","19.6 (UR)","(11.8 - 30.6) (UR)","23.6 (UR)","(14.0 - 37.2) (UR)"
"27157","Wabasha","Emergency department visits","2016-2018","All","All ages","181.9","63674","28.6","(24.4 - 32.7)","34.3","(29.2 - 39.4)"
"27159","Wadena","Emergency department visits","2016-2018","All","All ages","134.8","40191","33.5","(27.9 - 39.2)","36.9","(30.4 - 43.3)"
"27161","Waseca","Emergency department visits","2016-2018","All","All ages","232.5","55427","42.0","(36.6 - 47.3)","44.1","(38.3 - 49.9)"
"27163","Washington","Emergency department visits","2016-2018","All","All ages","1484.1","759716","19.5","(18.5 - 20.5)","20.9","(19.8 - 21.9)"
"27165","Watonwan","Emergency department visits","2016-2018","All","All ages","117.0","32243","36.3","(29.7 - 42.9)","38.6","(31.4 - 45.7)"
"27167","Wilkin","Emergency department visits","2016-2018","All","All ages","68.0","18517","36.7","(28.5 - 46.5)","40.3","(31.0 - 51.4)"
"27169

### Step 3: Fix Field Labels

Taking a look at existing header -- notice that the County field name (2nd column) is missing.

In [14]:
#Taking a look at header
!head '/content/MN_Asthma_Counties.csv'

fips,indicator,year,sex,age,count,population,rate,ci,ageAdjust,aci 
"27000","Minnesota","Emergency department visits","2016-2018","All","All ages","53812.0","16702743","32.2","(31.9 - 32.5)","34.0","(33.7 - 34.3)"
"27001","Aitkin","Emergency department visits","2016-2018","All","All ages","139.9","46342","30.2","(25.2 - 35.2)","40.9","(33.6 - 48.2)"
"27003","Anoka","Emergency department visits","2016-2018","All","All ages","2830.7","1040812","27.2","(26.2 - 28.2)","29.1","(28.0 - 30.2)"
"27005","Becker","Emergency department visits","2016-2018","All","All ages","257.6","100514","25.6","(22.5 - 28.8)","29.2","(25.6 - 32.9)"
"27007","Beltrami","Emergency department visits","2016-2018","All","All ages","362.2","137669","26.3","(23.6 - 29.0)","28.0","(25.0 - 31.0)"
"27009","Benton","Emergency department visits","2016-2018","All","All ages","267.5","118983","22.5","(19.8 - 25.2)","23.3","(20.5 - 26.2)"
"27011","Big Stone","Emergency department visits","2016-2018","All","All ages","9.3","146

In [56]:
#Removing the existing field names
%%bash
sed -i -e 1d '/content/MN_Asthma_Counties.csv'

In [57]:
#Add in the new field names on line 1
%%bash
sed -i '1i fips,county,indicator,year,sex,age,counts,population,rate,ci,ageAdjust,aci'  '/content/MN_Asthma_Counties.csv'

In [58]:
#Taking a look at updated header 
!head '/content/MN_Asthma_Counties.csv'

fips,county,indicator,year,sex,age,counts,population,rate,ci,ageAdjust,aci
"27000","Minnesota","Emergency department visits","2016-2018","All","All ages","53812.0","16702743","32.2","(31.9 - 32.5)","34.0","(33.7 - 34.3)"
"27001","Aitkin","Emergency department visits","2016-2018","All","All ages","139.9","46342","30.2","(25.2 - 35.2)","40.9","(33.6 - 48.2)"
"27003","Anoka","Emergency department visits","2016-2018","All","All ages","2830.7","1040812","27.2","(26.2 - 28.2)","29.1","(28.0 - 30.2)"
"27005","Becker","Emergency department visits","2016-2018","All","All ages","257.6","100514","25.6","(22.5 - 28.8)","29.2","(25.6 - 32.9)"
"27007","Beltrami","Emergency department visits","2016-2018","All","All ages","362.2","137669","26.3","(23.6 - 29.0)","28.0","(25.0 - 31.0)"
"27009","Benton","Emergency department visits","2016-2018","All","All ages","267.5","118983","22.5","(19.8 - 25.2)","23.3","(20.5 - 26.2)"
"27011","Big Stone","Emergency department visits","2016-2018","All","All ages","9.



---



---



---



---



### Step #4: Load Data into Python

In [25]:
#Load the pandas package
import pandas as pd

In [59]:
#Use read_table to read in the tab delimited file into Python
MN_Asthma = pd.read_csv('/content/MN_Asthma_Counties.csv') 

In [51]:
#Looking at first 5 records
MN_Asthma.tail(n=5)

,fips,county,indicator,year,sex,age,count,population,rate,ci,ageAdjust,aci
83,27165,Watonwan,Emergency department visits,2016-2018,All,All ages,117.0,32243,36.3,(29.7 - 42.9),38.6,(31.4 - 45.7)
84,27167,Wilkin,Emergency department visits,2016-2018,All,All ages,68.0,18517,36.7,(28.5 - 46.5),40.3,(31.0 - 51.4)
85,27169,Winona,Emergency department visits,2016-2018,All,All ages,369.1,150360,24.5,(22.0 - 27.1),29.0,(25.8 - 32.1)
86,27171,Wright,Emergency department visits,2016-2018,All,All ages,670.2,399395,16.8,(15.5 - 18.1),17.4,(16.0 - 18.7)
87,27173,Yellow Medicine,Emergency department visits,2016-2018,All,All ages,81.2,28968,28.0,(22.3 - 34.8),32.1,(25.3 - 40.2)


Next, install the dfply package that can be used to invoke various data verbs in Python

In [29]:
pip install dfply

     |████████████████████████████████| 614kB 6.8MB/s 


In [34]:
#Load the dfply package
from dfply import *

### Step #5: SELECT Desired Columns

In [62]:
##Using dfply to get groups, summarize, and split columns by State
MN_Asthma_SELECT = (
                    MN_Asthma 
                    >> select(X.fips, X.county, X.counts, X.population, X.rate)
                  )

#Pretty print the desired table
print(MN_Asthma_SELECT.to_string(index=False))

  fips             county   counts  population       rate
 27000          Minnesota  53812.0    16702743       32.2
 27001             Aitkin    139.9       46342       30.2
 27003              Anoka   2830.7     1040812       27.2
 27005             Becker    257.6      100514       25.6
 27007           Beltrami    362.2      137669       26.3
 27009             Benton    267.5      118983       22.5
 27011          Big Stone      9.3       14619   6.4 (UR)
 27013         Blue Earth    552.0      198426       27.8
 27015              Brown    128.4       74072       17.3
 27017            Carlton    356.8      105337       33.9
 27019             Carver    348.5      303487       11.5
 27021               Cass    241.2       86278       28.0
 27023           Chippewa     84.6       35270       24.0
 27025            Chisago    394.7      163911       24.1
 27027               Clay    505.6      188308       26.9
 27029         Clearwater     78.0       25971       30.0
 27031        

### Step #6: Write out the final table (csv file)

In [65]:
#Use read_table to read in the tab delimited file into Python
MN_Asthma_SELECT.to_csv('/content/MN_Asthma_after_SELECT.csv', sep=',', encoding='utf-8', index=False)

**Note**: The desired data file can be downloaded from the content folder -- expand the folder near the upper left side of screen to view its contents.



---



---



---

